In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tensorflow as tf

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Dense, Activation, Dropout, Flatten, SimpleRNN
from keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/641152877.py", line 2, in <module>
    from keras.utils import np_utils
ModuleNotFoundError: No module named 'keras'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_ge

In [2]:
import warnings
warnings.filterwarnings("ignore")

ERROR! Session/line number was not unique in database. History logging moved to new session 35


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/1408506528.py", line 1, in <module>
    from google.colab import drive
ModuleNotFoundError: No module named 'google.colab'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fi

In [3]:
DATA_DIRECTORY = ""

ERROR! Session/line number was not unique in database. History logging moved to new session 36


# Normalizing Data

In [4]:
def get_working_directory():
    return os.path.join(os.getcwd(), DATA_DIRECTORY)

get_working_directory()

'C:\\Users\\himan\\Documents\\jupyter_notebooks\\NSL-KDD\\'

In [5]:
def read_file_lines(dataset, filename):
    filepath = os.path.join(get_working_directory(), dataset, filename)
    with open(filepath, 'r', encoding='utf-8') as content:
        return content.readlines()

cc = read_file_lines('nsl', 'KDDTrain+.txt')[:5]
cc

['0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20\n',
 '0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.00,0.00,0.00,0.00,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15\n',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.00,1.00,0.00,0.00,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19\n',
 '0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.20,0.20,0.00,0.00,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21\n',
 '0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.00,0.00,0.00,0.00,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21\n']

In [6]:
def extract_features(a_line):
    return [x.strip() for x in a_line.split(',')]

print(extract_features(cc[0]))

['0', 'tcp', 'ftp_data', 'SF', '491', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '2', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '150', '25', '0.17', '0.03', '0.17', '0.00', '0.00', '0.00', '0.05', '0.00', 'normal', '20']


Numericalize features based on data analytics (Aurélien Géron Book)

In [7]:
def numericalize_feature(feature, protocol_type, service, flag):
    protocol_type_count = len(protocol_type)
    service_count = len(service)
    flag_count = len(flag)

    second_index = int(protocol_type_count + 1)
    third_index = int(protocol_type_count + service_count + 1)
    forth_index = int(protocol_type_count + service_count + flag_count + 1)

    # index 1 is protocol_type
    feature[1:1] = protocol_type[feature[1]]
    feature.pop(second_index)

    # index 2 + protocol_type_count is service
    feature[second_index:second_index] = service[feature[second_index]]
    feature.pop(third_index)
    # # index 3 + protocol_type_count + service_count is flag
    feature[third_index:third_index] = flag[feature[third_index]]
    feature.pop(forth_index)

    # make all values np.float64
    feature = [np.float64(x) for x in feature]

    return np.array(feature)

In [8]:
def numericalize_result(result, attack, attack_dict):
    second_index = int(1)
    # index 0 is attack
    result[0:0] = attack[attack_dict[result[0]]]
    result.pop(second_index)
    # make all values np.float64
    result = [np.float64(x) for x in result]
    return np.array(result)

In [9]:
def normalize_value(value, min, max):
    value = np.float64(value)
    min = np.float64(min)
    max = np.float64(max)
    if min == np.float64(0) and max == np.float64(0):
        return np.float64(0)
    result = np.float64((value - min) / (max - min))
    return result

In [10]:
def nsl_process_data_multiclass():
    """
    read from data folder and return a list
    [train_data, train_results, test_data, test_results]
    """
    train_data = read_file_lines('nsl', 'KDDTrain+.txt')
    test_data = read_file_lines('nsl', 'KDDTest+.txt')

    # create np.arrays of np.arrays from lines
    raw_train_data_features = [
        extract_features(x) for x in train_data]
    raw_test_data_features = [extract_features(x) for x in test_data]

    # train data: put index 0 to 40 in data, 41 and 42 into result (we
    # don't need 41,42 for now)
    raw_train_data_results = [x[41:42] for x in raw_train_data_features]
    raw_train_data_features = [x[0:41] for x in raw_train_data_features]

    # test data: put index 0 to 40 in data, 41 and 42 into result (we
    # don't need 41,42 for now)
    raw_test_data_results = [x[41:42] for x in raw_test_data_features]
    raw_test_data_features = [x[0:41] for x in raw_test_data_features]

    # stage 1 : numericalization --> index 1, 2 and 3 of dataset
    # 1.1 extract all protocol_types, services and flags
    protocol_type = dict()
    service = dict()
    flag = dict()
    attack = dict()
    attack_dict = {
        'normal': 'normal',
        'back': 'DoS',
        'land': 'DoS',
        'neptune': 'DoS',
        'pod': 'DoS',
        'smurf': 'DoS',
        'teardrop': 'DoS',
        'mailbomb': 'DoS',
        'apache2': 'DoS',
        'processtable': 'DoS',
        'udpstorm': 'DoS',
        'ipsweep': 'Probe',
        'nmap': 'Probe',
        'portsweep': 'Probe',
        'satan': 'Probe',
        'mscan': 'Probe',
        'saint': 'Probe',
        'ftp_write': 'R2L',
        'guess_passwd': 'R2L',
        'imap': 'R2L',
        'multihop': 'R2L',
        'phf': 'R2L',
        'spy': 'R2L',
        'warezclient': 'R2L',
        'warezmaster': 'R2L',
        'sendmail': 'R2L',
        'named': 'R2L',
        'snmpgetattack': 'R2L',
        'snmpguess': 'R2L',
        'xlock': 'R2L',
        'xsnoop': 'R2L',
        'worm': 'R2L',
        'buffer_overflow': 'U2R',
        'loadmodule': 'U2R',
        'perl': 'U2R',
        'rootkit': 'U2R',
        'httptunnel': 'U2R',
        'ps': 'U2R',
        'sqlattack': 'U2R',
        'xterm': 'U2R'
    }
    for entry in raw_train_data_features:
        protocol_type[entry[1]] = ""
        service[entry[2]] = ""
        flag[entry[3]] = ""

    for entry in raw_test_data_features:
        protocol_type[entry[1]] = ""
        service[entry[2]] = ""
        flag[entry[3]] = ""

    for entry in raw_train_data_results:
        attack[attack_dict[entry[0]]] = ""

    for entry in raw_test_data_results:
        attack[attack_dict[entry[0]]] = ""

    keys = list(protocol_type.keys())
    for i in range(0, len(keys)):
        protocol_type[keys[i]] = [int(d) for d in str(bin(i)[2:].zfill(len(protocol_type)))]

    keys = list(service.keys())
    for i in range(0, len(keys)):
        service[keys[i]] = [int(d) for d in str(bin(i)[2:].zfill(len(service)))]

    keys = list(flag.keys())
    for i in range(0, len(keys)):
        flag[keys[i]] = [int(d) for d in str(bin(i)[2:].zfill(len(flag)))]

    keys = list(attack.keys())
    for i in range(0, len(keys)):
        attack[keys[i]] = [int(i)]

    # train data
    numericalized_train_data_features = [numericalize_feature(x, protocol_type, service, flag) for x in raw_train_data_features]
    normalized_train_data_features = np.array(numericalized_train_data_features)

    numericalized_train_data_results = [numericalize_result(x, attack, attack_dict) for x in raw_train_data_results]
    normalized_train_data_results = np.array(numericalized_train_data_results)

    # test data
    numericalized_test_data_features = [numericalize_feature(x, protocol_type, service, flag) for x in raw_test_data_features]
    normalized_test_data_features = np.array(numericalized_test_data_features)

    numericalized_test_data_results = [numericalize_result(x, attack, attack_dict) for x in raw_test_data_results]
    normalized_test_data_results = np.array(numericalized_test_data_results)

    # stage 2: normalization --> x = (x - MIN) / (MAX - MIN) --> based
    # on columns

    # train data
    ymin_train = np.amin(numericalized_train_data_features, axis=0)
    ymax_train = np.amax(numericalized_train_data_features, axis=0)

    # test data
    ymin_test = np.amin(numericalized_test_data_features, axis=0)
    ymax_test = np.amax(numericalized_test_data_features, axis=0)

    # normalize train
    for x in range(0, normalized_train_data_features.shape[0]):
        for y in range(0, normalized_train_data_features.shape[1]):
            normalized_train_data_features[x][y] = normalize_value(normalized_train_data_features[x][y], ymin_train[y], ymax_train[y])

    # normalize test
    for x in range(0, normalized_test_data_features.shape[0]):
        for y in range(0, normalized_test_data_features.shape[1]):
            normalized_test_data_features[x][y] = normalize_value(normalized_test_data_features[x][y], ymin_test[y], ymax_test[y])

    mul_nsl = os.path.join(
        get_working_directory(), 'mul-nsl')
    if not os.path.exists(mul_nsl): 
        os.makedirs(mul_nsl)
    # save all normalized data
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_train_data_features.csv")
    np.savetxt(filepath, normalized_train_data_features, delimiter=",")
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_train_data_results.csv")
    np.savetxt(filepath, normalized_train_data_results, delimiter=",")
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_test_data_features.csv")
    np.savetxt(filepath, normalized_test_data_features, delimiter=",")
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_test_data_results.csv")
    np.savetxt(filepath, normalized_test_data_results, delimiter=",")

    return True

In [11]:
def nsl_data_multiclass():
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_train_data_features.csv")
    if (not os.path.isfile(filepath)):
        nsl_process_data_multiclass()
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_train_data_features.csv")
    normalized_train_data_features = np.loadtxt(filepath, delimiter=",")
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_train_data_results.csv")
    normalized_train_data_results = np.loadtxt(filepath, delimiter=",")
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_test_data_features.csv")
    normalized_test_data_features = np.loadtxt(filepath, delimiter=",")
    filepath = os.path.join(get_working_directory(), 'mul-nsl', "normalized_test_data_results.csv")
    normalized_test_data_results = np.loadtxt(filepath, delimiter=",")

    return [normalized_train_data_features, normalized_train_data_results, normalized_test_data_features, normalized_test_data_results]

According to paper, protocol_type, flags and service are most relevant features

In [12]:
x_train, y_train, x_test, y_test = nsl_data_multiclass()

# reshape input to be [samples, timesteps, features]
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])

In [ ]:
# label here
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/2087251135.py", line 2, in <module>
    y_train = np_utils.to_categorical(y_train)
NameError: name 'np_utils' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_geti

In [13]:
x_train[:1]

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000

ERROR! Session/line number was not unique in database. History logging moved to new session 37


# Simple Deep Neural Network

In [14]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# define deep neural network
dnn_model = Sequential()
dnn_model.add(Dense(128, input_shape=(x_train.shape[1], x_train.shape[2])))
dnn_model.add(Dropout(0.1))
dnn_model.add(Dense(256))
dnn_model.add(Dropout(0.1))
dnn_model.add(Dense(128))
dnn_model.add(Dropout(0.1))
dnn_model.add(Flatten())
dnn_model.add(Dense(5))
dnn_model.add(Activation('softmax'))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3456023917.py", line 2, in <module>
    dnn_model = Sequential()
NameError: name 'Sequential' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, 

In [ ]:
dnn_model.summary()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/2068600966.py", line 1, in <module>
    dnn_model.summary()
NameError: name 'dnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number

In [ ]:
# set training on GPU
# tf.device('/device:GPU:0')

start = time.time()
# decide epochs
epochs = 10

# optimizer
adam = Adam(lr=0.0005)
dnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
dnn_model.fit(x_train, y_train, validation_data=(x_train, y_train), epochs=epochs, batch_size=32)

# save the model
dnn_model.save(f"dnn_model.hdf5")

# metrics for model
print("--- %s seconds ---" % (time.time() - start))
loss, accuracy = dnn_model.evaluate(x_test, y_test, batch_size=32)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy * 100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/1412477227.py", line 10, in <module>
    dnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
NameError: name 'dnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", 

In [ ]:
y_pred = dnn_model.predict(x_test)
y_pred = np.round(y_pred).astype(int)
print("Anomaly in Test: ", np.count_nonzero(y_test, axis=0))
print("Anomaly in Prediction: ", np.count_nonzero(y_pred, axis=0))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3800377652.py", line 1, in <module>
    y_pred = dnn_model.predict(x_test)
NameError: name 'dnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerfra

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3800377652.py", line 1, in <module>
    y_pred = dnn_model.predict(x_test)
NameError: name 'dnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_nodes
    if (await self

In [ ]:
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/190521116.py", line 1, in <module>
    print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
NameError: name 'confusion_matrix' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in 

  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3186, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3396, in run_ast_nodes
    self.showtraceback()
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2080, in showtraceback
    value, tb, tb_offset=tb_offset)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1368, in structured_traceback
    self, etype, value, tb, tb_offset, number_of_lines_of_context)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1268, in structured_traceback
    self, etype, value, tb, tb_offset, number_of_lines_of_context
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\I

In [ ]:
print(classification_report(y_test, y_pred))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/580191101.py", line 1, in <module>
    print(classification_report(y_test, y_pred))
NameError: name 'classification_report' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    retu

# Simple RNN model

In [15]:
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

ERROR! Session/line number was not unique in database. History logging moved to new session 43


In [ ]:
# define recurrent neural network
rnn_model = Sequential()
rnn_model.add(SimpleRNN(120, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
rnn_model.add(Dropout(0.2))
rnn_model.add(SimpleRNN(120, return_sequences=True))
rnn_model.add(Dropout(0.2))
rnn_model.add(SimpleRNN(120, return_sequences=False))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(5))
rnn_model.add(Activation('softmax'))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/889180453.py", line 2, in <module>
    rnn_model = Sequential()
NameError: name 'Sequential' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, n

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/889180453.py", line 2, in <module>
    rnn_model = Sequential()
NameError: name 'Sequential' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_nodes
    if (await self.run_code(

In [ ]:
rnn_model.summary()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/4272429008.py", line 1, in <module>
    rnn_model.summary()
NameError: name 'rnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 1502, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 1460, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 696, in getsourcefile
    if getattr(getmodule(object, filename), '__loader__', None) is not None:
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 733, in getmodule
    if ismodule(module) and hasattr(module, '__file__'):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\l

In [ ]:
# set training on GPU
# tf.device('/device:GPU:0')

start = time.time()
# decide epochs
epochs = 10

# optimizer
adam = Adam(lr=0.0005)
rnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(x_train, y_train, validation_data=(x_train, y_train), epochs=epochs, batch_size=32)

# save the model
rnn_model.save(f"rnn_model.hdf5")

# metrics for model
print("--- %s seconds ---" % (time.time() - start))
loss, accuracy = rnn_model.evaluate(x_test, y_test, batch_size=32)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy * 100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/117966231.py", line 10, in <module>
    rnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
NameError: name 'rnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", l

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/117966231.py", line 10, in <module>
    rnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
NameError: name 'rnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshe

In [ ]:
y_pred = rnn_model.predict(x_test)
y_pred = np.round(y_pred).astype(int)
print("Anomaly in Test: ", np.count_nonzero(y_test, axis=0))
print("Anomaly in Prediction: ", np.count_nonzero(y_pred, axis=0))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3216946822.py", line 1, in <module>
    y_pred = rnn_model.predict(x_test)
NameError: name 'rnn_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerfra

In [ ]:
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/190521116.py", line 1, in <module>
    print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
NameError: name 'confusion_matrix' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/190521116.py", line 1, in <module>
    print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
NameError: name 'confusion_matrix' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 

In [ ]:
print(classification_report(y_test, y_pred))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/580191101.py", line 1, in <module>
    print(classification_report(y_test, y_pred))
NameError: name 'classification_report' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    retu

# Long Short-Term Memory

In [16]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

ERROR! Session/line number was not unique in database. History logging moved to new session 49


In [ ]:
# define lstm neural network
lstm_model = Sequential()
lstm_model.add((LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(128, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(128, return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(5))
lstm_model.add(Activation('softmax'))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/4143331748.py", line 2, in <module>
    lstm_model = Sequential()
NameError: name 'Sequential' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb,

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/4143331748.py", line 2, in <module>
    lstm_model = Sequential()
NameError: name 'Sequential' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_nodes
    if (await self.run_cod

In [ ]:
lstm_model.summary()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3910366260.py", line 1, in <module>
    lstm_model.summary()
NameError: name 'lstm_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, numb

In [ ]:
# set training on GPU
# tf.device('/device:GPU:0')

start = time.time()
# decide epochs
epochs = 10

# optimizer
adam = Adam(lr=0.0001)
lstm_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(x_train, y_train, validation_data=(x_train, y_train), epochs=epochs, batch_size=32)

# save the model
lstm_model.save(f"lstm_model.hdf5")

# metrics for model
print("--- %s seconds ---" % (time.time() - start))
loss, accuracy = lstm_model.evaluate(x_test, y_test, batch_size=32)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy * 100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3426775730.py", line 10, in <module>
    lstm_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
NameError: name 'lstm_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/3426775730.py", line 10, in <module>
    lstm_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
NameError: name 'lstm_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactive

In [ ]:
y_pred = lstm_model.predict(x_test)
y_pred = np.round(y_pred).astype(int)
print("Anomaly in Test: ", np.count_nonzero(y_test, axis=0))
print("Anomaly in Prediction: ", np.count_nonzero(y_pred, axis=0))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/1214942871.py", line 1, in <module>
    y_pred = lstm_model.predict(x_test)
NameError: name 'lstm_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 281, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 1502, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 1460, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\inspect.py", line 696, in getsourcefile

In [ ]:
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/190521116.py", line 1, in <module>
    print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
NameError: name 'confusion_matrix' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ModuleNotFoundError: No module named 'tensorflow_core.estimator'
Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/190521116.py", line 1, in <module>
    print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
NameError: name 'confusion_matrix' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorfl

In [ ]:
print(classification_report(y_test, y_pred))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/580191101.py", line 1, in <module>
    print(classification_report(y_test, y_pred))
NameError: name 'classification_report' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    retu

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\himan\AppData\Local\Temp/ipykernel_14312/580191101.py", line 1, in <module>
    print(classification_report(y_test, y_pred))
NameError: name 'classification_report' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\himan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_nod